In [1]:
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from tensorflow.python.keras.layers import *
from tensorflow.python.keras.models import Model
import numpy as np 
import pandas as pd 
import re
import nltk
from preprocess import *
from models import *

2022-12-03 00:34:51.737654: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 00:34:53.732243: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-03 00:34:53.732423: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2022-12-03 00:34:53.732441: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] 

In [2]:
df = pd.read_csv("questions.csv")
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [3]:
# question_1, question_2 = df['question1'].to_list(), df['question2'].to_list()
# is_duplicate = df['is_duplicate'].to_list()
# q1_preprocessed, q2_preprocessed = preprocess_neural(question_1, question_2, is_duplicate)

In [4]:
df1 = pd.read_csv("preprocessed_neural.csv")
q1_preprocessed, q2_preprocessed, is_duplicate = df1['question1'].to_list(), df1['question2'].to_list(), df1['is_duplicate'].to_list()

In [5]:
df1.head()

,question1,question2,is_duplicate
0,step step guide invest share market india,step step guide invest share market,0
1,story kohinoor kohinoor diamond,would happen indian government stole kohinoor ...,0
2,increase speed internet connection using vpn,internet speed increased hacking dns,0
3,mentally lonely solve,find remainder math2324math divided 2423,0
4,one dissolve water quikly sugar salt methane c...,fish would survive salt water,0


Acquired Test data

In [6]:
MAX_NB_WORDS = 200000
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)
tokenizer.fit_on_texts(list(df1['question1'].values.astype(str))+list(df1['question2'].values.astype(str)))

In [7]:

q1_sequence = tokenizer.texts_to_sequences(df['question1'].values.astype(str))
q1_sequence = pad_sequences(q1_sequence, maxlen = 30, padding='post')

q2_sequence = tokenizer.texts_to_sequences(df['question2'].values.astype(str))
q2_sequence = pad_sequences(q2_sequence, maxlen = 30, padding='post')

In [8]:
windex = tokenizer.word_index

In [9]:
embedding_index = {}
with open('glove.6B.300d.txt','r') as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[1:], 'float32')
        embedding_index[word] = vectors
    f.close()

In [10]:
embedding_matrix = np.random.random((len(windex)+1, 300))

for word, i in windex.items():
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape)

(108101, 300)


In [11]:
q1_embeddings = []
for i in range(len(q1_sequence)):
    embedding = np.zeros(300)
    for j in range(len(q1_sequence[i])):
        embedding += embedding_matrix[q1_sequence[i][j]]
    embedding /= len(q1_sequence[i])
    q1_embeddings.append(embedding)

q2_embeddings = []
for i in range(len(q2_sequence)):
    embedding = np.zeros(300)
    for j in range(len(q2_sequence[i])):
        embedding += embedding_matrix[q2_sequence[i][j]]
    embedding /= len(q2_sequence[i])
    q2_embeddings.append(embedding)

In [12]:
# change all 0 to 0 1 and 1 to 1 0
for i in range(len(is_duplicate)):
    if is_duplicate[i] == 0:
        is_duplicate[i] = [0,1]
    else:
        is_duplicate[i] = [1,0]

In [13]:
#split the data into 70-20-10 train-validation-test with random state 42
from sklearn.model_selection import train_test_split
q1_train, q1_test, q2_train, q2_test, y_train, y_test = train_test_split(q1_embeddings, q2_embeddings, is_duplicate, test_size=0.1, random_state=42)
q1_train, q1_val, q2_train, q2_val, y_train, y_val = train_test_split(q1_train, q2_train, y_train, test_size=0.222, random_state=42)

In [14]:
#print the ratio of positive and negative samples in train, validation and test
y_train, y_val, y_test = np.array(y_train), np.array(y_val), np.array(y_test)
print("Train: ", sum(y_train)/len(y_train))
print("Validation: ", sum(y_val)/len(y_val))
print("Test: ", sum(y_test)/len(y_test))

Train:  [0.36970243 0.63029757]
Validation:  [0.36774353 0.63225647]
Test:  [0.36907706 0.63092294]


In [15]:
def concatenate_embeddings(q1_embeddings, q2_embeddings):
    embeddings = np.zeros((len(q1_embeddings), 900))
    for i in range(len(q1_embeddings)):
        for j in range(0, 300):
            embeddings[i][j] = q1_embeddings[i][j] + q2_embeddings[i][j]
        for j in range(300, 600):
            embeddings[i][j] = q1_embeddings[i][j-300] - q2_embeddings[i][j-300]
        for j in range(600, 900):
            embeddings[i][j] = q1_embeddings[i][j-600] * q2_embeddings[i][j-600]
    return embeddings
xtrain_concat = concatenate_embeddings(q1_train, q2_train)
xval_concat = concatenate_embeddings(q1_val, q2_val)
xtest_concat = concatenate_embeddings(q1_test, q2_test)

In [16]:
print(xval_concat.shape)
print(xtrain_concat.shape)

(80790, 900)
(283125, 900)


In [22]:
model = CBOW(embedding_matrix, embedding_matrix.shape[0], loss="binary_crossentropy", epochs=150)

In [23]:
model.fit(xtrain_concat, xval_concat, y_train, y_val)

In [24]:
model.get_model_summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 300)               270300    
                                                                 
 dropout (Dropout)           (None, 300)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               60200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 100)               20100     
                                                                 
 dropout_2 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 2)                

In [31]:
model.model.fit(xtrain_concat, y_train, epochs=150, batch_size=32, validation_batch_size=16, validation_data=(xval_concat, y_val))

Epoch 1/150
4424/4424 [==============================] - 43s 9ms/step - loss: 0.6159 - accuracy: 0.6393 - val_loss: 0.5905 - val_accuracy: 0.6801
Epoch 2/150
4424/4424 [==============================] - 43s 10ms/step - loss: 0.5879 - accuracy: 0.6732 - val_loss: 0.5772 - val_accuracy: 0.6896
Epoch 3/150
4424/4424 [==============================] - 43s 10ms/step - loss: 0.5754 - accuracy: 0.6850 - val_loss: 0.5671 - val_accuracy: 0.6936
Epoch 4/150
4424/4424 [==============================] - 41s 9ms/step - loss: 0.5656 - accuracy: 0.6920 - val_loss: 0.5579 - val_accuracy: 0.6912
Epoch 5/150
4424/4424 [==============================] - 42s 9ms/step - loss: 0.5582 - accuracy: 0.6983 - val_loss: 0.5661 - val_accuracy: 0.6928
Epoch 6/150
4424/4424 [==============================] - 42s 10ms/step - loss: 0.5522 - accuracy: 0.7026 - val_loss: 0.5524 - val_accuracy: 0.6993
Epoch 7/150
4424/4424 [==============================] - 42s 9ms/step - loss: 0.5491 - accuracy: 0.7038 - val_loss: 0.535

In [32]:
y_pred = model.predict(xtest_concat)
y_pred1d = []
for i in range(len(y_pred)):
    if(y_pred[i][0] > y_pred[i][1]):
        y_pred1d.append(0)
    else:
        y_pred1d.append(1)

y_test1d = []
for i in range(len(y_test)):
    if(y_test[i][0] > y_test[i][1]):
        y_test1d.append(0)
    else:
        y_test1d.append(1)

from sklearn.metrics import accuracy_score, f1_score
print("Accuracy: ", accuracy_score(y_test1d, y_pred1d))
print("F1 Score: ", f1_score(y_test1d, y_pred1d))

1264/1264 [==============================] - 4s 3ms/step
Accuracy:  0.8065268572559105
F1 Score:  0.8336609241916214
